Short example script for calculating $\Delta(k_\rm{p},z=1.0) = k_\rm{p}^3 P_{\rm{lin}}(k_\rm{p},z=1.0)$ at pivot scale $k_\rm{p} = 0.2\, h/\rm{Mpc}$:

In [6]:
from classy import Class
import numpy as np

#pivot scale + redshift
z_p = 1.0
k_p = 0.2 #our pivot scale, in h/Mpc units

#settings to configure CLASS computations
settings = {
    'output': 'mPk', #which quantities we want CLASS to compute
    'z_pk' : z_p,
    'non linear' : 'none',
    'P_k_max_1/Mpc':1.0 # CLASS uses 1/Mpc units
}


In [2]:
def compute_norm(params):
    #initialize Class instance and set parameters
    cosmo = Class();
    cosmo.set(settings);
    cosmo.set(params)
    cosmo.compute()
    #print(cosmo.get_current_derived_parameters(['A_s']))

    #if CLASS used shooting method to determine A_s, check that it worked to better that 0.1% in sigma8
    if 'sigma8' in params:
        delta_sigma8 = np.abs((params['sigma8']-cosmo.sigma(8/cosmo.h(),0))/params['sigma8'])
        assert delta_sigma8 < 1.e-3, f"A_s shooting method caused %.4f fractional bias in sigma8"%(delta_sigma8)

    h = cosmo.h()
    k_p_CLASS = k_p*h # convert to 1/Mpc units used by CLASS
    norm = k_p**3*cosmo.pk(k_p_CLASS,z_p)*h**3 #factor h**3 to convert P(k) from 1/Mpc to h/Mpc units
    return norm

### Parameter parsing: Abacus parameterization $\omega_\mathrm{b},\omega_\mathrm{cdm},\sigma_8,n_\mathrm{s}$

In [7]:
# flat LCDM version *with theta_s constraint*

#Abacus parameter vector p = [omega_b,omega_cdm,sigma8,n_s]
def compute_norm_Abacus(p): 
    Abacus_LCDM ={
        'omega_b': p[0],
        'omega_cdm': p[1],
        'sigma8': p[2], 
        'n_s': p[3], 
        'N_ur': 2.0328,
        'N_ncdm' : 1.0,
        'omega_ncdm' : 0.0006442,
        '100*theta_s' : 1.041533, #from Summit paper
        'recombination' : 'HyRec',
        'tau_reio' : 0.0544
    }
    return compute_norm(Abacus_LCDM)

#Abacus parameter vector p = [omega_b,omega_cdm,sigma8,n_s]
p =[0.022370000000000004, 0.12000000000000001, 0.807952, 0.9649]
compute_norm_Abacus(p)

5.709395057620228